In [1]:
import os
import re
from typing import List
from pathlib import Path
from langchain.docstore.document import Document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import MarkdownHeaderTextSplitter

In [2]:
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker
from langchain_ollama import OllamaEmbeddings
from typing import List


# fixed_model_name = "deepseek-coder:6.7b"
# fixed_model_name = "exaone3.5:latest"
# fixed_model_name = "mistral:latest"
# fixed_model_name = "llama3:8b"
fixed_model_name = "nomic-embed-text:latest"

# markdown_file_path = "data/dev_center_guide_allmd_touched.md"
markdown_file_path = "data/dev_center_guide_allmd.md"

vdb_file_path = "data/fiass_semantic_chunker_v06_" + fixed_model_name[:3] + ".db"

def test_semantic_chunker() -> list[Document]:
    """SemanticChunker를 사용한 마크다운 파일의 의미적 분할을 테스트합니다."""
    
    try:
        # 파일 내용 읽기
        with open(markdown_file_path, 'r', encoding='utf-8') as f:
            markdown_content = f.read()
        
        print(f"✅ 마크다운 파일 로드 완료: {len(markdown_content)} 문자")
        
        # 임베딩 모델 설정
        print("🔄 임베딩 모델 로딩 중...")
        embeddings = OllamaEmbeddings(model=fixed_model_name)
        
        # SemanticChunker 설정
        print("🔄 SemanticChunker 설정 중...")
        semantic_chunker = SemanticChunker(
            embeddings=embeddings,
            breakpoint_threshold_type="percentile",
            breakpoint_threshold_amount=80, # 95
            min_chunk_size=100,
        )
        
        print("✅ SemanticChunker 설정 완료")
        
        # 의미적 분할 수행
        print("🔄 마크다운 문서를 의미적으로 분할 중...")
        print("⚠️  이 과정은 시간이 걸릴 수 있습니다...")
        semantic_chunks = semantic_chunker.split_text(markdown_content)
        
        print(f"✅ 분할 완료: {len(semantic_chunks)}개의 청크 생성")
        
        # Document 객체 생성
        documents = []
        for i, chunk in enumerate(semantic_chunks):
            # 메타데이터 구성
            metadata = {
                'source': markdown_file_path,
                'chunk_id': i,
                'chunk_type': 'semantic_chunker',
                'content_length': len(chunk),
                'chunker_model': 'SemanticChunker',
                'embedding_model': 'exaone3.5:latest',
            }
            
            # Document 객체 생성
            doc = Document(
                page_content=chunk,
                metadata=metadata
            )
            
            documents.append(doc)
        
        print(f"✅ Document 객체 생성 완료: {len(documents)}개")
        
        return documents
            
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        return list()
    
def print_stats(documents: List[Document]):
    total_chars = sum(len(doc.page_content) for doc in documents)
    avg_chars = total_chars / len(documents)
    
    print(f"\n📊 === 통계 정보 ===")
    print(f"총 문서 수: {len(documents)}개")
    print(f"총 문자 수: {total_chars:,}자")
    print(f"평균 청크 크기: {avg_chars:.1f}자")
    
    # 청크 크기 통계
    chunk_sizes = [len(doc.page_content) for doc in documents]
    print(f"최소 청크 크기: {min(chunk_sizes)}자")
    print(f"최대 청크 크기: {max(chunk_sizes)}자")
    
    # 첫 번째 문서 샘플 출력
    print(f"\n📄 === 첫 번째 Document 샘플 ===")
    first_doc = documents[0]
    print(f"메타데이터: {first_doc.metadata}")
    print(f"내용 (처음 150자): {first_doc.page_content[:150]}...")
    
    
def embed_and_save(docs: List[Document], output_path: str):
    """문서를 임베딩하고 FAISS 데이터베이스로 저장합니다."""
    # 임베딩 모델 초기화
    embedding_model = OllamaEmbeddings(model=fixed_model_name)
    
    # FAISS 데이터베이스 생성 및 저장
    db = FAISS.from_documents(docs, embedding_model)
    db.save_local(output_path)
    print(f"✅ 임베딩 저장 완료: {output_path}")

print("🚀 SemanticChunker 테스트 시작...")
docs = test_semantic_chunker()
print_stats(docs)
embed_and_save(docs, vdb_file_path)



print("success ..")

🚀 SemanticChunker 테스트 시작...
✅ 마크다운 파일 로드 완료: 383039 문자
🔄 임베딩 모델 로딩 중...
🔄 SemanticChunker 설정 중...
✅ SemanticChunker 설정 완료
🔄 마크다운 문서를 의미적으로 분할 중...
⚠️  이 과정은 시간이 걸릴 수 있습니다...
✅ 분할 완료: 182개의 청크 생성
✅ Document 객체 생성 완료: 182개

📊 === 통계 정보 ===
총 문서 수: 182개
총 문자 수: 379,971자
평균 청크 크기: 2087.8자
최소 청크 크기: 112자
최대 청크 크기: 20122자

📄 === 첫 번째 Document 샘플 ===
메타데이터: {'source': 'data/dev_center_guide_allmd.md', 'chunk_id': 0, 'chunk_type': 'semantic_chunker', 'content_length': 569, 'chunker_model': 'SemanticChunker', 'embedding_model': 'exaone3.5:latest'}
내용 (처음 150자): 출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21.md
# 원스토어 인앱결제 API V7(SDK V21) 연동 안내 및 다운로드

원스토어의 최신 인앱결제 API V7(SDK V21)이 출시되었습니다. 보다 강력하고...
✅ 임베딩 저장 완료: data/fiass_semantic_chunker_v06_nom.db
success ..


In [3]:
cnt = 0
for doc in docs:
    if 'PNS' in doc.page_content:
        print(f"-------- doc_index: {cnt} --------")
        print(doc.metadata)
        print(doc.page_content)
        print("-" * 100)
        cnt += 1

print(f"PNS 문서 개수: {cnt}")

-------- doc_index: 0 --------
{'source': 'data/dev_center_guide_allmd.md', 'chunk_id': 1, 'chunk_type': 'semantic_chunker', 'content_length': 1659, 'chunker_model': 'SemanticChunker', 'embedding_model': 'exaone3.5:latest'}
원스토어 인앱결제 개요](v21/ov)
* [02. 원스토어 인앱결제 적용을 위한 사전준비](v21/pre)
* [03. 결제 테스트 및 보안](v21/test)
* [04. 원스토어 인앱결제 SDK를 사용해 구현하기](v21/sdk)
* [05. 원스토어 인앱결제 레퍼런스](v21/references)
* [06. 원스토어 인앱결제 서버 API (API V7)](v21/serverapi)
* [07. PNS(Payment Notification Service) 이용하기](v21/pns)
* [08. 정기 결제 적용하기](v21/subs)
* [09. 원스토어 인앱결제 릴리즈 노트](v21/releasenote)
* [10. Sample App Download](v21/sample)
* [11. V21로 원스토어 인앱결제 업그레이드 하기](v21/upgrade)
* [12. Unity에서 원스토어 인앱결제 (SDK V21) 사용하기](v21/unity)
* [13. Unity에서 IAP SDK v21로 업그레이드 하기](v21/unitymig)
* [14. Flutter에서 원스토어 인앱결제 사용하기](v21/flutter)
* [15. 웹 결제 규격 적용하기](v21/web-payment)

\


출처: https://onestore-dev.gitbook.io/dev/tools/billing/v21/ov.md
# 01. 원스토어 인앱결제 개요

## **원스토어 인앱결제란?** <a href="#id-01." id="id-01."></a>

원스토어 인앱결제(In

In [9]:
###### Retriever Check from FAISS Vector DB ######

from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import BM25Retriever


# ✅ 3. 임베딩 모델 초기화 (Ollama)
embedding_model = OllamaEmbeddings(model=fixed_model_name)

# 저장된 데이터를 로드
loaded_db = FAISS.load_local(
    folder_path=vdb_file_path,
    # index_name="index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True,
)

retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 10, "fetch_k": 30, "lambda_mult": 0.7}
    # search_type="similarity",
    # search_kwargs={"k": 50}
)

bm25 = BM25Retriever.from_documents(
    docs,
    bm25_params={"k1": 1.2, "b": 0.75}
)

bm25.k = 20

ensembled_retriever = EnsembleRetriever(
    retrievers=[bm25, retriever],
    weights=[0.5, 0.5]
)

res = bm25.invoke("PNS의 메세지 형식은 어떻게 되나요?")

# res = retriever.invoke(
#     # "PNS(Payment Notification Service)의 메세지 규격은 어떻게 되나요?"
#     # "원스토어 PNS(Payment Notification Service)의 json 메세지 형식을 알려주세요"
#     # "PNS의 purcahseState는 어떤 값이 있나요?"
#     "원스토어의 PNS(Payment Notification Service)에서 사용되는 purcahseState는 어떤 값이 있나요?"
# )

# res = retriever.invoke(
#     "원스토어 인앱결제의 PNS의 개념을 설명해주세요"
# )

print(f"검색된 문서 수: {len(res)}")

idx = 0
for doc in res: 
    print(f"--- doc_index: {idx} ---")
    print(doc.page_content)  # Print first 100 characters of each document
    # print(doc.metadata)
    print('-' * 40)
    idx += 1

검색된 문서 수: 20
--- doc_index: 0 ---
아래 내용을 참고하시기 바랍니다. *   **Response Body :** JSON 형식

    | Element Name | Data Type | Data Size | Description |
    | ------------ | --------- | --------- | ----------- |
    | code         | String    | -         | 응답 코드       |
    | message      | String    | -         | 응답 메시지      |
    | error        | Object    | -         | <p><br></p> |
*   **Example**

    ```json
    HTTP/1.1 400 Bad Request
    Content-type: application/json;charset=UTF-8
    {
        "error" : {
            "code" : "NoSuchData",
            "message" : "The requested data could not be found."
        }
    }
    ```

## **공통 코드**  <a href="#id-06.-api-apiv7" id="id-06.-api-apiv7"></a>

### **상품타입 코드**  <a href="#id-06.-api-apiv7" id="id-06.-api-apiv7"></a>

| Code         | Name   | Description     |
| ------------ | ------ | --------------- |
| inapp        | 관리형 상품 | 소비성/영구성/기간제 상품  |
| auto         | 월정액 상품 | 월 자동결제 상품       |
| subscription | 구독형 상품 | 구독형(자동결제) 상품    

In [29]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# RAG 체인 구성 요소
# embedding_model = OllamaEmbeddings(model=fixed_model_name)
llm = ChatOllama(model=fixed_model_name, temperature=0.3)

# 검색기 설정
retriever = loaded_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 3, "fetch_k": 10, "lambda_mult": 0.9}
)
# retriever = create_multi_query_retriever(loaded_db, llm)
# retriever = create_enhanced_ensemble_retriever(loaded_db, docs_markdown, llm)

# 프롬프트 템플릿
prompt_template = """
당신은 원스토어 In App Purchase(인앱결제) 연동 전문가입니다. 학습한 문서를 바탕으로 질문에 답변해주세요.

답변시 다음 사항을 고려해주세요:
1. 컨텍스트에 명시된 정보만 사용하세요
2. 구체적인 값이나 코드 예시가 있다면 포함하세요
3. 불확실한 정보는 "컨텍스트에 해당 정보가 없습니다"라고 답변하세요

컨텍스트:
{context}

질문: {question}

답변:"""

prompt = PromptTemplate.from_template(prompt_template)

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인이 생성되었습니다!")

✅ RAG 체인이 생성되었습니다!


In [31]:
# 테스트 질문들
test_questions = [
    # "원스토어 인앱결제 연동을 위한 전반적인 절차를 알려주세요",
    # "정기결제 구현 방법을 알려주세요",
    # "결제 검증은 어떻게 하나요?",
    # "관리형 상품과 구독형 상품의 차이점은?",
    # "원스토어의 PNS(Payment Notification Service)란 무엇인가요?"
    # "PNS의 메시지 수신 서버의 관점에서 Java SpringFramework로 구현한 코드를 예제를 알려주세요",
    # "원스토어 PNS(Payment Notification Service)의 json 메세지 형식을 알려주세요",
    "원스토어의 PNS(Payment Notification Service)에서 결제 원스토어가 개발사 서버로 전송하는 메세지중 purcahseState는 어떤 값이 있나요?"
]

print("🤖 RAG 체인 테스트 시작\n")
print("=" * 80)

for i, question in enumerate(test_questions, 1):
    print(f"\n📝 질문 {i}: {question}")
    print("-" * 60)
    
    try:
        answer = rag_chain.invoke(question)
        print(answer)
    except Exception as e:
        print(f"❌ 오류 발생: {str(e)}")
    
    print("=" * 80)

🤖 RAG 체인 테스트 시작


📝 질문 1: 원스토어의 PNS(Payment Notification Service)에서 결제 원스토어가 개발사 서버로 전송하는 메세지중 purcahseState는 어떤 값이 있나요?
------------------------------------------------------------
원스토어의 PNS(Payment Notification Service)에서 결제 상태를 나타내는 `purchaseState` 매개변수는 다음과 같은 값을 가질 수 있습니다:

- **SUCCESS**: 결제가 성공적으로 완료되었음을 나타냅니다.
- **FAIL**: 결제 과정에서 실패가 발생하였음을 나타냅니다.
- **CANCEL**: 결제가 취소되었음을 나타냅니다.
- **PENDING**: 결제 상태가 아직 결정되지 않았음을 나타냅니다 (예: 결제 중).

이러한 상태 값들은 결제 프로세스의 다양한 단계와 결과를 개발자에게 전달하는데 사용됩니다. 정확한 값은 결제 프로세스의 특정 시점에 따라 달라질 수 있으므로, 실제 구현 시에는 해당 시점의 정확한 상태 값을 확인해야 합니다.
